## Objectives

- Establish meaningful grouping of address by clustering DEX traders and LP
    - Feature Extraction:
        - with or without balances at swap time (web3py fetch balance at block)
    - Dimensionality reduction:
        - UMAP / tSNE or PCA / ICA
    - Clustering:
        - DBSCAN
        - silhouette evaluation
    **Deliverable**
    - Visualization:
        - scatter plot with color-coded returns? (TBD)

**Secondary objective: identify which group has the most profitable activity**
- Triggered Average of price by swap in/out by group of addresses

- Predict the return of the next candle based on the "history" of activity of the groups of addresses
    - Feature extracion
        - 

In [1]:
from py0xcluster.utils import web3_utils, query_utils
from py0xcluster.main_classes.classes import *

In [ ]:
etherscan_API_path = r'C:\Users\phar0732\Documents\ID\etherscan\etherscan_api.csv'
etherscan_token = str(pd.read_csv(etherscan_API_path, header=None).values[0][0])
etherscan_token

address0 = '0x424a31f5caa3155437ebed0ce80169639f6689d5'
address1 = '0x728BD56A965dC3B9aDA745301C2f731599d2fe13'


In [3]:
web3_utils.get_nonce('0x424a31f5caa3155437ebed0ce80169639f6689d5')

1

### Define the query based on the univ2 graph

In [2]:


#SAND_trades = TradesDEX('SAND-ETH', 'start', 'end', subgraph_url) # get name and set from it

start_date = (2022, 11, 6, 00, 00, 00)
end_date = (2022, 11, 7, 00, 00, 00)

params = {
    # 'max_rows': 50,
    # 'skip': 0,
    'pair_address': '0xce84867c3c02b05dc570d0135103d3fb9cc19433',
    'timestamp_start': query_utils.timestamp_tuple_to_unix(start_date),
    'timestamp_end': query_utils.timestamp_tuple_to_unix(end_date)
    }
import time
 
# assigned regular string date

# print("date_time =>", timestamp_tuple_to_unix(start_date))

SAND_trades = TradesDEX('SAND-ETH') # get name and set from it

sand_df = SAND_trades.get_swaps('uni2', params) # TODO: implement start and end date
#trades.raw_df.loc[trades.raw_df['amount0In']==0]
#print(type(trades.raw_df['amount0Out'].values))
#sushiGraphQL.json['data']

sand_df

outputs: 48 trades


,id,timestamp,sender,to,amount0In,amount1In,amount0Out,amount1Out,logIndex,amountUSD,transaction.id,pair.token0.symbol,pair.token0.id,pair.token1.symbol,pair.token1.id,pd_datetime
0,0xb652e8ab0135a28b97fb2a236ff0b48aee45f580437b...,1667777015,0xbfef411d9ae30c5b471d529c838f1abb7b65d67f,0xbfef411d9ae30c5b471d529c838f1abb7b65d67f,0.000000,0.412270,376.562503,0.000000,44,650.917458,0xb652e8ab0135a28b97fb2a236ff0b48aee45f580437b...,SUSHI,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-11-06 23:23:35
1,0xa8a66b16960bc58dad7865d6dfbdb959caad4204da9e...,1667775971,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,300.000000,0.000000,0.000000,0.326265,272,521.598344,0xa8a66b16960bc58dad7865d6dfbdb959caad4204da9e...,SUSHI,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-11-06 23:06:11
2,0x406ba2eb3065d6d920ace984869b23386e3a0cac14fc...,1667775107,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xdef1c0ded9bec7f1a1670819833240f027b25eff,44.066081,0.000000,0.000000,0.048066,104,76.934230,0x406ba2eb3065d6d920ace984869b23386e3a0cac14fc...,SUSHI,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-11-06 22:51:47
3,0x099ba3eef8c5c0967a59d1b46903e108d85a415c5ae9...,1667774387,0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0,0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0,0.000000,0.183955,167.817009,0.000000,27,293.697546,0x099ba3eef8c5c0967a59d1b46903e108d85a415c5ae9...,SUSHI,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-11-06 22:39:47
4,0xb4c927715b437d9bae8f34b16d7be53a55886e9a6fec...,1667774363,0xa57bd00134b2850b2a1c55860c9e9ea100fdd6cf,0x56178a0d5f301baf6cf3e1cd53d9863437345bf9,540.332402,0.000000,0.000000,0.590641,458,943.000866,0xb4c927715b437d9bae8f34b16d7be53a55886e9a6fec...,SUSHI,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-11-06 22:39:23
5,0xaadcc0e1b63f3e8578356ab6acfe65ed9520e1a4b673...,1667774219,0x0eae044f00b0af300500f090ea00027097d03000,0xa425833329fa3a1f722230eb0999160b917e4ce9,0.000000,0.079635,72.124706,0.000000,14,127.145644,0xaadcc0e1b63f3e8578356ab6acfe65ed9520e1a4b673...,SUSHI,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-11-06 22:36:59
6,0x757700c44d47f6c8deaffa4264367c3d603bf266b5db...,1667774207,0xa57bd00134b2850b2a1c55860c9e9ea100fdd6cf,0x56178a0d5f301baf6cf3e1cd53d9863437345bf9,988.623768,0.000000,0.000000,1.093692,199,1746.198645,0x757700c44d47f6c8deaffa4264367c3d603bf266b5db...,SUSHI,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-11-06 22:36:47
7,0x8766cadc972ff5a989b1e1910d9e1ad4fc1165560f97...,1667773127,0xa57bd00134b2850b2a1c55860c9e9ea100fdd6cf,0x56178a0d5f301baf6cf3e1cd53d9863437345bf9,529.737391,0.000000,0.000000,0.593811,340,948.293364,0x8766cadc972ff5a989b1e1910d9e1ad4fc1165560f97...,SUSHI,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-11-06 22:18:47
8,0xa9ce814e855fe8b1f6fd5210f42492137286f8ae917d...,1667772455,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,562.203740,0.000000,0.000000,0.629620,17,1011.697661,0xa9ce814e855fe8b1f6fd5210f42492137286f8ae917d...,SUSHI,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-11-06 22:07:35
9,0x79d5a2eb69774aaaeb908daa4b27085922fc2bbfc87d...,1667772455,0x0000e0ca771e21bd00057f54a68c30d400000000,0x0000e0ca771e21bd00057f54a68c30d400000000,0.000000,0.673377,597.489345,0.000000,27,1082.009332,0x79d5a2eb69774aaaeb908daa4b27085922fc2bbfc87d...,SUSHI,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-11-06 22:07:35


### 

In [26]:

sender_sum = sand_df.groupby('sender').agg(sum).sort_values('amountUSD', ascending=False)
sender_mean = sand_df.groupby('sender').agg({'amountUSD': 'mean'}).sort_values('amountUSD', ascending=False)
sender_mean.head(50)

,amountUSD
sender,
0x00000000003b3cc22af3ae1eac0440bcee416b40,125418.786580
0x1d6e8bac6ea3730825bde4b005ed7b2b39a2932d,88492.586700
0x000000000035b5e5ad9019092c665357240f594e,79049.298200
0x000000005736775feb0c8568e7dee77222a26880,54934.228807
0x00000000032962b51589768828ad878876299e14,51669.335950
0x7da08ac2740268f36634c2579739be26a1235958,26819.790846
0xeb5d3a91b5e721fa251e03bda2c9578e90f30f84,24723.757924
0x4c46ff32fcf1fcd25bae2e26ed5fd3a682412ffc,24481.306109
0x0000000000d41c96294ccdac8612bdfe29c641af,23805.741808
